# False Positives Analysis

**Definitions:**
- **False Positive** = Lean verification passed but answer is wrong
- **Gaming** = False positive where model "proved" True/False (not Failure/Uncertain)
- **Conservative** = False positive where model said Failure when it should have proved

Excluding stories 368 & 435 (cases 75, 76, 77, 156, 157, 158, 159) due to contradictory premises.

In [12]:
import json
import pandas as pd

exclude_cases = {75, 76, 77, 156, 157, 158, 159}

def load_results(path):
    with open(path) as f:
        return [json.loads(l) for l in f]

baseline = load_results('../results/simplelean/gpt-5_folio_baseline_20251223_095234/results.jsonl')
bidir_true = load_results('../results/simplelean/gpt-5_folio_bidir_true_20251224_123917/results.jsonl')
bidir_false = load_results('../results/simplelean/gpt-5_folio_bidir_false_20251224_125254/results.jsonl')

In [13]:
def analyze(results, name, exclude=exclude_cases):
    filtered = [r for r in results if r['case_idx'] not in exclude]
    
    total = len(filtered)
    correct = sum(1 for r in filtered if r['correct'])
    lean_pass = sum(1 for r in filtered if r.get('lean_verification') and r['lean_verification'].get('success', False))
    
    # False positives: Lean pass AND wrong answer
    fp = []
    for r in filtered:
        lean_ok = r.get('lean_verification') and r['lean_verification'].get('success', False)
        if lean_ok and not r['correct']:
            fp.append({
                'case': r['case_idx'],
                'pred': r['prediction'],
                'gt': r['ground_truth']
            })
    
    return {
        'name': name,
        'total': total,
        'accuracy': f"{correct}/{total} ({100*correct/total:.1f}%)",
        'lean_pass': f"{lean_pass}/{total} ({100*lean_pass/total:.1f}%)",
        'false_positives': len(fp),
        'fp_rate': f"{100*len(fp)/total:.1f}%",
        'fp_details': fp
    }

In [14]:
results = [
    analyze(baseline, 'Baseline'),
    analyze(bidir_true, 'bidir_true'),
    analyze(bidir_false, 'bidir_false')
]

summary_df = pd.DataFrame([{
    'Condition': r['name'],
    'Accuracy': r['accuracy'],
    'Lean Pass': r['lean_pass'],
    'False Positives': r['false_positives'],
    'FP Rate': r['fp_rate']
} for r in results])

summary_df

,Condition,Accuracy,Lean Pass,False Positives,FP Rate
0,Baseline,160/196 (81.6%),178/196 (90.8%),19,9.7%
1,bidir_true,182/196 (92.9%),196/196 (100.0%),14,7.1%
2,bidir_false,184/196 (93.9%),195/196 (99.5%),12,6.1%


## False Positives by Error Type

- **Gaming**: Proved wrong answer (True when gt=False/Uncertain, or False when gt=True/Uncertain)
- **Conservative**: Said Failure/Uncertain when could have proved

In [15]:
def categorize_fps(fp_details, condition):
    gaming = []
    conservative = []
    
    for fp in fp_details:
        pred, gt = fp['pred'], fp['gt']
        
        if condition == 'Baseline':
            if pred == 'True' and gt != 'True':
                gaming.append(fp)
            else:
                conservative.append(fp)
        elif condition == 'bidir_true':
            if pred == 'True' and gt != 'True':
                gaming.append(fp)
            else:  # Failure when gt=True
                conservative.append(fp)
        elif condition == 'bidir_false':
            if pred == 'False' and gt != 'False':
                gaming.append(fp)
            else:  # Failure when gt=False
                conservative.append(fp)
    
    return gaming, conservative

for r in results:
    gaming, conservative = categorize_fps(r['fp_details'], r['name'])
    print(f"\n{r['name']}:")
    print(f"  Gaming: {len(gaming)}")
    print(f"  Conservative: {len(conservative)}")


Baseline:
  Gaming: 5
  Conservative: 14

bidir_true:
  Gaming: 4
  Conservative: 10

bidir_false:
  Gaming: 0
  Conservative: 12


## Detailed False Positives

In [16]:
print("=" * 60)
print("BASELINE FALSE POSITIVES")
print("=" * 60)
for fp in sorted(results[0]['fp_details'], key=lambda x: x['case']):
    error_type = "GAMING" if fp['pred'] == 'True' and fp['gt'] != 'True' else "CONSERVATIVE"
    print(f"Case {fp['case']:3d}: {fp['pred']:>10} -> {fp['gt']:<10} [{error_type}]")

BASELINE FALSE POSITIVES
Case  36:  Uncertain -> True       [CONSERVATIVE]
Case  40:  Uncertain -> True       [CONSERVATIVE]
Case  41:       True -> False      [GAMING]
Case  46:  Uncertain -> True       [CONSERVATIVE]
Case  70:       True -> Uncertain  [GAMING]
Case  83:       True -> False      [GAMING]
Case  89:       True -> Uncertain  [GAMING]
Case 118:  Uncertain -> True       [CONSERVATIVE]
Case 119:  Uncertain -> False      [CONSERVATIVE]
Case 122:  Uncertain -> False      [CONSERVATIVE]
Case 123:  Uncertain -> True       [CONSERVATIVE]
Case 127:  Uncertain -> False      [CONSERVATIVE]
Case 128:  Uncertain -> True       [CONSERVATIVE]
Case 130:  Uncertain -> True       [CONSERVATIVE]
Case 140:  Uncertain -> False      [CONSERVATIVE]
Case 165:  Uncertain -> True       [CONSERVATIVE]
Case 196:  Uncertain -> False      [CONSERVATIVE]
Case 197:  Uncertain -> True       [CONSERVATIVE]
Case 202:       True -> Uncertain  [GAMING]


In [17]:
print("=" * 60)
print("BIDIR_TRUE FALSE POSITIVES")
print("=" * 60)
for fp in sorted(results[1]['fp_details'], key=lambda x: x['case']):
    error_type = "GAMING" if fp['pred'] == 'True' else "CONSERVATIVE"
    print(f"Case {fp['case']:3d}: {fp['pred']:>10} -> {fp['gt']:<10} [{error_type}]")

BIDIR_TRUE FALSE POSITIVES
Case   5:    Failure -> True       [CONSERVATIVE]
Case  36:    Failure -> True       [CONSERVATIVE]
Case  40:    Failure -> True       [CONSERVATIVE]
Case  46:    Failure -> True       [CONSERVATIVE]
Case  70:       True -> Uncertain  [GAMING]
Case  83:       True -> False      [GAMING]
Case  89:       True -> Uncertain  [GAMING]
Case  99:    Failure -> True       [CONSERVATIVE]
Case 118:    Failure -> True       [CONSERVATIVE]
Case 123:    Failure -> True       [CONSERVATIVE]
Case 128:    Failure -> True       [CONSERVATIVE]
Case 130:    Failure -> True       [CONSERVATIVE]
Case 141:    Failure -> True       [CONSERVATIVE]
Case 202:       True -> Uncertain  [GAMING]


In [18]:
print("=" * 60)
print("BIDIR_FALSE FALSE POSITIVES")
print("=" * 60)
for fp in sorted(results[2]['fp_details'], key=lambda x: x['case']):
    error_type = "GAMING" if fp['pred'] == 'False' else "CONSERVATIVE"
    print(f"Case {fp['case']:3d}: {fp['pred']:>10} -> {fp['gt']:<10} [{error_type}]")

BIDIR_FALSE FALSE POSITIVES
Case  41:    Failure -> False      [CONSERVATIVE]
Case  83:    Failure -> False      [CONSERVATIVE]
Case  91:    Failure -> False      [CONSERVATIVE]
Case  92:    Failure -> False      [CONSERVATIVE]
Case 100:    Failure -> False      [CONSERVATIVE]
Case 102:    Failure -> False      [CONSERVATIVE]
Case 106:    Failure -> False      [CONSERVATIVE]
Case 114:    Failure -> False      [CONSERVATIVE]
Case 119:    Failure -> False      [CONSERVATIVE]
Case 122:    Failure -> False      [CONSERVATIVE]
Case 127:    Failure -> False      [CONSERVATIVE]
Case 140:    Failure -> False      [CONSERVATIVE]


## Gaming Cases Comparison

Cases where model "proved" wrong answer across conditions

In [19]:
# Find gaming cases for each condition
baseline_gaming = {fp['case'] for fp in results[0]['fp_details'] if fp['pred'] == 'True' and fp['gt'] != 'True'}
bidir_true_gaming = {fp['case'] for fp in results[1]['fp_details'] if fp['pred'] == 'True'}
bidir_false_gaming = {fp['case'] for fp in results[2]['fp_details'] if fp['pred'] == 'False'}

print("Gaming cases:")
print(f"  Baseline:    {sorted(baseline_gaming)}")
print(f"  bidir_true:  {sorted(bidir_true_gaming)}")
print(f"  bidir_false: {sorted(bidir_false_gaming)}")
print()
print(f"Overlap (baseline & bidir_true): {sorted(baseline_gaming & bidir_true_gaming)}")

Gaming cases:
  Baseline:    [41, 70, 83, 89, 202]
  bidir_true:  [70, 83, 89, 202]
  bidir_false: []

Overlap (baseline & bidir_true): [70, 83, 89, 202]


In [20]:
# Load FOLIO data for ground truth
with open('../data/folio/original/folio-validation.json') as f:
    folio_data = json.load(f)

# Gaming cases from bidir_true
gaming_cases = [70, 83, 89, 202]

# Load as dict for easy lookup
baseline_dict = {r['case_idx']: r for r in baseline}
bidir_true_dict = {r['case_idx']: r for r in bidir_true}
bidir_false_dict = {r['case_idx']: r for r in bidir_false}

print("=" * 70)
print("GAMING CASES COMPARISON (bidir_true gaming cases)")
print("=" * 70)
print(f"{'Case':<6} {'GT':<12} {'Baseline':<12} {'bidir_true':<12} {'bidir_false':<12} {'Story'}")
print("-" * 70)

for idx in gaming_cases:
    gt = folio_data[idx]['label']
    story = folio_data[idx].get('story_id', '?')
    
    bl = baseline_dict[idx]['prediction']
    bt = bidir_true_dict[idx]['prediction']
    bf = bidir_false_dict[idx]['prediction']
    
    bl_mark = "ok" if baseline_dict[idx]['correct'] else "X"
    bt_mark = "ok" if bidir_true_dict[idx]['correct'] else "X"
    bf_mark = "ok" if bidir_false_dict[idx]['correct'] else "X"
    
    print(f"{idx:<6} {gt:<12} {bl+' '+bl_mark:<12} {bt+' '+bt_mark:<12} {bf+' '+bf_mark:<12} {story}")

GAMING CASES COMPARISON (bidir_true gaming cases)
Case   GT           Baseline     bidir_true   bidir_false  Story
----------------------------------------------------------------------
70     Uncertain    True X       True X       Failure ok   322
83     False        True X       True X       Failure X    306
89     Uncertain    True X       True X       Failure ok   58
202    Uncertain    True X       True X       Failure ok   101


## Case 83 Deep Dive

Case 83 is interesting: GT=False, but bidir_true said True and bidir_false said Failure (both wrong!)

The model proved True via **ex falso quodlibet** (vacuous truth).

In [21]:
import re

# Show case 83 premises and conclusion
print("=" * 70)
print("CASE 83: PROBLEM")
print("=" * 70)
print(f"Premises:\n{folio_data[83]['premises']}\n")
print(f"Conclusion: {folio_data[83]['conclusion']}")
print(f"Ground Truth: {folio_data[83]['label']}")

CASE 83: PROBLEM
Premises:
If a restaurant is listed in Yelp’s recommendations, then the restaurant has not received many negative reviews.
All restaurants with a rating greater than four are listed in Yelp’s recommendations.
Some restaurants that do not provide take-out service receive many negative reviews.
All restaurants that are popular among local residents have ratings greater than four.
The Hamden Plaza Subway store has a rating greater than four, or it is popular among local residents.

Conclusion: If the Hamden Plaza Subway store provides take-out service and receives many negative reviews, then its rating is greater than 4 and it does not provide take-out service.
Ground Truth: False


In [22]:
# Show bidir_true Lean code for case 83
with open('../results/simplelean/gpt-5_folio_bidir_true_20251224_123917/responses/case_83.txt') as f:
    content = f.read()

lean_match = re.search(r'<lean>(.*?)</lean>', content, re.DOTALL)
print("=" * 70)
print("CASE 83: bidir_true LEAN CODE")
print("=" * 70)
print(lean_match.group(1).strip() if lean_match else "No Lean code found")

CASE 83: bidir_true LEAN CODE
axiom obj : Type
axiom H : obj

axiom Listed : obj → Prop
axiom ManyNegative : obj → Prop
axiom RatingGT4 : obj → Prop
axiom TakeOut : obj → Prop
axiom Popular : obj → Prop

-- If a restaurant is listed in Yelp’s recommendations, then it has not received many negative reviews.
axiom A1 : ∀ x : obj, Listed x → ¬ ManyNegative x
-- All restaurants with a rating greater than four are listed in Yelp’s recommendations.
axiom A2 : ∀ x : obj, RatingGT4 x → Listed x
-- Some restaurants that do not provide take-out service receive many negative reviews.
axiom A3 : ∃ x : obj, ¬ TakeOut x ∧ ManyNegative x
-- All restaurants that are popular among local residents have ratings greater than four.
axiom A4 : ∀ x : obj, Popular x → RatingGT4 x
-- The Hamden Plaza Subway store has a rating greater than four, or it is popular among local residents.
axiom A5 : RatingGT4 H ∨ Popular H

-- Statement to evaluate:
-- If the Hamden Plaza Subway store provides take-out service and 

## Gaming Cases May Actually Be Dataset Issues

| Case | GT | Model | Actual Status |
|------|-----|-------|---------------|
| 70 | Uncertain | True | Need to check |
| 83 | False | True | **Model may be correct** (vacuous truth) |
| 89 | Uncertain | True | Need to check |
| 202 | Uncertain | True | Need to check |

## Case 70 Analysis (GT: Uncertain, Model: True)

In [23]:
# Case 70 - GT: Uncertain, Model: True
print("=" * 70)
print("CASE 70: PROBLEM")
print("=" * 70)
print(f"Premises:\n{folio_data[70]['premises']}\n")
print(f"Conclusion: {folio_data[70]['conclusion']}")
print(f"Ground Truth: {folio_data[70]['label']}")

CASE 70: PROBLEM
Premises:
All growth stocks are bought to earn profits from rapid price appreciation.
If the stock price is bought to earn profits from rapid price appreciation, then it is not suitable for a retirement fund.
Some stocks are growth stocks.
All mature stocks are suitable for a retirement fund.
KO is a mature stock.

Conclusion: KO is a stock.
Ground Truth: Uncertain


In [24]:
# Case 70 bidir_true Lean code
with open('../results/simplelean/gpt-5_folio_bidir_true_20251224_123917/responses/case_70.txt') as f:
    content = f.read()

lean_match = re.search(r'<lean>(.*?)</lean>', content, re.DOTALL)
print("=" * 70)
print("CASE 70: bidir_true LEAN CODE")
print("=" * 70)
print(lean_match.group(1).strip() if lean_match else "No Lean code found")

CASE 70: bidir_true LEAN CODE
axiom obj : Type
axiom KO : obj

axiom Stock : obj → Prop
axiom Growth : obj → Prop
axiom Mature : obj → Prop
axiom BoughtRapid : obj → Prop
axiom SuitableRetirement : obj → Prop

-- All growth stocks are bought to earn profits from rapid price appreciation.
axiom A1 : ∀ x : obj, Growth x → BoughtRapid x
-- If a stock is bought to earn profits from rapid price appreciation, then it is not suitable for a retirement fund.
axiom A2 : ∀ x : obj, BoughtRapid x → ¬ SuitableRetirement x
-- Some stocks are growth stocks.
axiom A3 : ∃ x : obj, Stock x ∧ Growth x
-- All mature stocks are suitable for a retirement fund.
axiom A4 : ∀ x : obj, Mature x → SuitableRetirement x
-- KO is a mature stock (hence: KO is mature and KO is a stock).
axiom A5_mature : Mature KO
axiom A5_stock : Stock KO

theorem KO_is_stock : Stock KO := by
  exact A5_stock


## Case 89 Analysis (GT: Uncertain, Model: True)

In [25]:
# Case 89 - GT: Uncertain, Model: True
print("=" * 70)
print("CASE 89: PROBLEM")
print("=" * 70)
print(f"Premises:\n{folio_data[89]['premises']}\n")
print(f"Conclusion: {folio_data[89]['conclusion']}")
print(f"Ground Truth: {folio_data[89]['label']}")

CASE 89: PROBLEM
Premises:
Books contain tons of knowledge.
When a person reads a book, that person gains knowledge. 
If a person gains knowledge, they become smarter.
Harry read the book “Walden” by Henry Thoreau.

Conclusion: A smarter person has gained knowledge.
Ground Truth: Uncertain


In [26]:
# Case 89 bidir_true Lean code
with open('../results/simplelean/gpt-5_folio_bidir_true_20251224_123917/responses/case_89.txt') as f:
    content = f.read()

lean_match = re.search(r'<lean>(.*?)</lean>', content, re.DOTALL)
print("=" * 70)
print("CASE 89: bidir_true LEAN CODE")
print("=" * 70)
print(lean_match.group(1).strip() if lean_match else "No Lean code found")

CASE 89: bidir_true LEAN CODE
axiom Person : Type
axiom Book : Type

axiom Reads : Person → Book → Prop
axiom Gained : Person → Prop
axiom Smarter : Person → Prop

axiom Harry : Person
axiom Walden : Book

-- When a person reads a book, that person gains knowledge.
axiom R1 : ∀ p : Person, ∀ b : Book, Reads p b → Gained p
-- If a person gains knowledge, they become smarter.
axiom R2 : ∀ p : Person, Gained p → Smarter p
-- Harry read Walden.
axiom T1 : Reads Harry Walden

-- Statement to evaluate: A smarter person has gained knowledge. (∃ person who is smarter and has gained knowledge)
theorem smarter_person_has_gained : ∃ p : Person, Smarter p ∧ Gained p :=
by
  have hG : Gained Harry := R1 Harry Walden T1
  have hS : Smarter Harry := R2 Harry hG
  exact Exists.intro Harry (And.intro hS hG)


## Case 202 Analysis (GT: Uncertain, Model: True)

In [27]:
# Case 202 - GT: Uncertain, Model: True
print("=" * 70)
print("CASE 202: PROBLEM")
print("=" * 70)
print(f"Premises:\n{folio_data[202]['premises']}\n")
print(f"Conclusion: {folio_data[202]['conclusion']}")
print(f"Ground Truth: {folio_data[202]['label']}")

CASE 202: PROBLEM
Premises:
Ailton Silva, born in 1995, is commonly known as Ailton.
Ailton is a football player who was loaned out to Braga.
Ailton Silva is a Brazilian footballer who plays for Náutico.
Náutico is a football club along with Braga.
Fluminense is a football club.

Conclusion: Ailton Silva was loaned out to a football club.
Ground Truth: Uncertain


In [28]:
# Case 202 bidir_true Lean code
with open('../results/simplelean/gpt-5_folio_bidir_true_20251224_123917/responses/case_202.txt') as f:
    content = f.read()

lean_match = re.search(r'<lean>(.*?)</lean>', content, re.DOTALL)
print("=" * 70)
print("CASE 202: bidir_true LEAN CODE")
print("=" * 70)
print(lean_match.group(1).strip() if lean_match else "No Lean code found")

CASE 202: bidir_true LEAN CODE
axiom obj : Type

axiom AiltonSilva : obj
axiom Ailton : obj
axiom Nautico : obj
axiom Braga : obj
axiom Fluminense : obj

axiom FootballPlayer : obj → Prop
axiom Brazilian : obj → Prop
axiom PlaysFor : obj → obj → Prop
axiom FootballClub : obj → Prop
axiom LoanedTo : obj → obj → Prop

-- Ailton Silva, commonly known as Ailton
axiom name_eq : Ailton = AiltonSilva

-- Ailton is a football player who was loaned out to Braga
axiom T_player : FootballPlayer Ailton
axiom T_loan : LoanedTo Ailton Braga

-- Ailton Silva is a Brazilian footballer who plays for Náutico
axiom T_brazil : Brazilian AiltonSilva
axiom T_player2 : FootballPlayer AiltonSilva
axiom T_plays : PlaysFor AiltonSilva Nautico

-- Náutico is a football club along with Braga; Fluminense is a football club
axiom club_nautico : FootballClub Nautico
axiom club_braga : FootballClub Braga
axiom club_flu : FootballClub Fluminense

theorem conclusion : ∃ c : obj, FootballClub c ∧ LoanedTo AiltonSilva c 

## Summary: Gaming Cases Analysis

After reviewing the Lean proofs for each gaming case:

| Case | GT | Issue | Verdict |
|------|-----|-------|---------|
| 70 | Uncertain | Model added `axiom A5_stock : Stock KO` not in premises! Premises say "KO is a mature stock" but model assumed "mature stock" implies "stock" | **GAMING** - invented axiom |
| 83 | False | Model proved True via **vacuous truth** - antecedent (TakeOut ∧ ManyNegative) is False, so implication is True | **Debatable** - technically correct logic |
| 89 | Uncertain | Valid chain: Harry read → gained → smarter → ∃ (Smarter ∧ Gained) | **Model correct** - valid reasoning |
| 202 | Uncertain | Braga is football club + Ailton loaned to Braga → ∃ club loaned to | **Model correct** - valid inference |

### Conclusion
- **1 true gaming case**: Case 70 (invented axiom)
- **1 debatable case**: Case 83 (vacuous truth)
- **2 dataset issues**: Cases 89, 202 (model's answer appears correct)